## Food Demand Forecasting Challenge(Practice Problem at analytics Vidhya)
 
* about : https://datahack.analyticsvidhya.com/contest/genpact-machine-learning-hackathon-1/#About) 
* data description : https://datahack.analyticsvidhya.com/contest/genpact-machine-learning-hackathon-1/#ProblemStatement
* forecasting target : num_orders
* Evaluation : 100 * RMSLE ( root of mean squared logarithmic error )


## 3. Feature Engineering  & Parameter tuning

In [155]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import lightgbm as lgb
from sklearn.model_selection import train_test_split,ParameterGrid
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

In [317]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')
df_info_meal = pd.read_csv('./data/meal_info.csv')
df_info_fulfil = pd.read_csv('./data/fulfilment_center_info.csv')

In [318]:
df_train = pd.merge(df_train, df_info_fulfil,
                    how="left",
                    left_on='center_id',
                    right_on='center_id')

df_train = pd.merge(df_train, df_info_meal,
                    how='left',
                    left_on='meal_id',
                    right_on='meal_id')

In [319]:
df_train.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,center_type,op_area,category,cuisine
0,1379560,1,55,1885,136.83,152.29,0,0,177,647,56,TYPE_C,2.0,Beverages,Thai
1,1466964,1,55,1993,136.83,135.83,0,0,270,647,56,TYPE_C,2.0,Beverages,Thai
2,1346989,1,55,2539,134.86,135.86,0,0,189,647,56,TYPE_C,2.0,Beverages,Thai
3,1338232,1,55,2139,339.50,437.53,0,0,54,647,56,TYPE_C,2.0,Beverages,Indian
4,1448490,1,55,2631,243.50,242.50,0,0,40,647,56,TYPE_C,2.0,Beverages,Indian


In [320]:
df_train.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,center_type,op_area,category,cuisine
0,1379560,1,55,1885,136.83,152.29,0,0,177,647,56,TYPE_C,2.0,Beverages,Thai
1,1466964,1,55,1993,136.83,135.83,0,0,270,647,56,TYPE_C,2.0,Beverages,Thai
2,1346989,1,55,2539,134.86,135.86,0,0,189,647,56,TYPE_C,2.0,Beverages,Thai
3,1338232,1,55,2139,339.50,437.53,0,0,54,647,56,TYPE_C,2.0,Beverages,Indian
4,1448490,1,55,2631,243.50,242.50,0,0,40,647,56,TYPE_C,2.0,Beverages,Indian


In [321]:
df_test = pd.merge(df_test, df_info_fulfil,
                   how="left",
                   left_on='center_id',
                   right_on='center_id')

df_test = pd.merge(df_test, df_info_meal,
                   how='left',
                   left_on='meal_id',
                   right_on='meal_id')

* (Feature Engineering) "base_price"와 "checkout_price"(할인 적용 가격)의 차이값 컬럼 생성

In [322]:
df_train['price_diff'] = df_train['base_price'] - df_train['checkout_price']
df_test['price_diff'] =  df_test['base_price'] - df_test['checkout_price']

* (Feature Engineering) "base_price"와 "checkout_price(할인 적용 가격)"의 percentage(%)기준 차이값 컬럼생성(price_diff_percent)

In [323]:
df_train['price_diff_percent'] = (df_train['base_price'] - df_train['checkout_price']) / df_train['base_price']
df_test['price_diff_percent'] =  (df_test['base_price'] - df_test['checkout_price']) / df_test['base_price']

* (Feature Engineering) 위에서 생성한 "price_diff_percent" 값을 기준으로 아래와 같이 네가지 값으로 구성된 범주형 컬럼 생성

1)  10% 이상 : "UP"

2)  5% ~ 10% : "sl_UP"

3) -5% ~ 5%  : "notchange"

4) -5% ~-10% : "sl_Down

5) -10% 이하 : "DOWN"


In [324]:
df_train.loc[df_train['price_diff_percent'] > 0.1, 'big_diff'] = "UP"
df_train.loc[(df_train['price_diff_percent'] < 0.1) & (df_train['price_diff_percent'] > 0.05), 'big_diff'] = "sl_UP"

df_train.loc[df_train['price_diff_percent'] < -0.1, 'big_diff'] = "DOWM"
df_train.loc[(df_train['price_diff_percent'] > -0.1) & (df_train['price_diff_percent'] < -0.05), 'big_diff'] = "sl_Down"

df_train['big_diff'] = df_train['big_diff'].fillna("notchange")


df_test.loc[df_test['price_diff_percent'] > 0.1, 'big_diff'] = "UP"
df_test.loc[(df_test['price_diff_percent'] < 0.1) & (df_test['price_diff_percent'] > 0.05), 'big_diff'] = "sl_UP"

df_test.loc[df_test['price_diff_percent'] < -0.1, 'big_diff'] = "DOWM"
df_test.loc[(df_test['price_diff_percent'] > -0.1) & (df_test['price_diff_percent'] < -0.05), 'big_diff'] = "sl_Down"

df_test['big_diff'] = df_test['big_diff'].fillna("notchange")

* (Feature Engineering) "week"컬럼값은 순차적으로 증가하는 형태로 표시. 주간단위로 발생하는 주기적/반복적인 "num_orders"의 변동을 학습시키기 위해 sin, cos 함수를 이용하여 cyclical 형태의 값으로 구성된 신규 컬럼을 생성

(참고링크) https://www.avanwyk.com/encoding-cyclical-features-for-deep-learning/

In [325]:
# 52주 -> 1년 주기

df_train['week_sin'] = np.sin(2 * np.pi * df_train['week'] / 52.143)
df_train['week_cos'] = np.cos(2 * np.pi * df_train['week'] / 52.143)

df_test['week_sin'] = np.sin(2 * np.pi * df_test['week'] / 52.143)
df_test['week_cos'] = np.cos(2 * np.pi * df_test['week'] / 52.143)

In [326]:
# 4주 -> 1개월 주기

df_train['week_sin_1'] = np.sin(2 * np.pi * df_train['week'] / 4)
df_train['week_cos_1'] = np.cos(2 * np.pi * df_train['week'] / 4)

df_test['week_sin_1'] = np.sin(2 * np.pi * df_test['week'] / 4)
df_test['week_cos_1'] = np.cos(2 * np.pi * df_test['week'] / 4)

* (Feature Engineering) 프로모션 여부를 표시한 컬럼인 "emailer_for_promotion", "homepage_featured"를 하나의 컬럼으로 생성

1) 프로모션을 진행하지 않았을경우  : 0

2) 프로모션을 둘중 한개만 진행했을 경우 : 1

3) 프로모션을 모두 진행했을 경우 : 2

In [327]:
df_train['email_plus_homepage'] = df_train['emailer_for_promotion'] + df_train['homepage_featured']

df_test['email_plus_homepage'] = df_test['emailer_for_promotion'] + df_test['homepage_featured']

* (Feature Engineering) "city_code"컬럼과 "region_code"컬럼을 합쳐 하나의 컬럼으로 생성

In [328]:
df_train['city_region'] = df_train['city_code'].astype('str') + '_' + df_train['region_code'].astype('str')

df_test['city_region'] = df_test['city_code'].astype('str') + '_' + df_test['region_code'].astype('str')

In [284]:
label_encode_columns = ['center_id', 
                        'meal_id', 
                        'email_plus_homepage',                          
                        'city_region',                        
                        'op_area',
                        'center_type',
                        'category',
                        'cuisine',
                        'big_diff',
                        'emailer_for_promotion',
                        'homepage_featured',
                        'city_code',
                        'region_code'
                        
                       ]

In [329]:
label_encode_columns

['center_id',
 'meal_id',
 'email_plus_homepage',
 'city_region',
 'op_area',
 'center_type',
 'category',
 'cuisine',
 'big_diff',
 'emailer_for_promotion',
 'homepage_featured',
 'city_code',
 'region_code']

In [330]:
df_train.columns

Index(['id', 'week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured', 'num_orders', 'city_code',
       'region_code', 'center_type', 'op_area', 'category', 'cuisine',
       'price_diff', 'price_diff_percent', 'big_diff', 'week_sin', 'week_cos',
       'week_sin_1', 'week_cos_1', 'email_plus_homepage', 'city_region'],
      dtype='object')

In [331]:
df_train['email_plus_homepage'].value_counts()

0    388874
1     48443
2     19231
Name: email_plus_homepage, dtype: int64

In [332]:
df_train['big_diff'].head()

0           UP
1    notchange
2    notchange
3           UP
4    notchange
Name: big_diff, dtype: object

In [333]:
le = preprocessing.LabelEncoder()

for col in label_encode_columns:
    le.fit(df_train[col])
    df_train[col + '_encoded'] = le.transform(df_train[col])
    df_test[col + '_encoded'] = le.transform(df_test[col])

In [334]:
feature_name = [col for col in df_train.columns if col not in label_encode_columns]

In [335]:
feature_name

['id',
 'week',
 'checkout_price',
 'base_price',
 'num_orders',
 'price_diff',
 'price_diff_percent',
 'week_sin',
 'week_cos',
 'week_sin_1',
 'week_cos_1',
 'center_id_encoded',
 'meal_id_encoded',
 'email_plus_homepage_encoded',
 'city_region_encoded',
 'op_area_encoded',
 'center_type_encoded',
 'category_encoded',
 'cuisine_encoded',
 'big_diff_encoded',
 'emailer_for_promotion_encoded',
 'homepage_featured_encoded',
 'city_code_encoded',
 'region_code_encoded']

* 학습에 불필요한 "id"컬럼과 target 컬럼인 "num_orders"를 제외

In [336]:
feature_name.remove('id')
feature_name.remove('num_orders')

In [337]:
feature_name

['week',
 'checkout_price',
 'base_price',
 'price_diff',
 'price_diff_percent',
 'week_sin',
 'week_cos',
 'week_sin_1',
 'week_cos_1',
 'center_id_encoded',
 'meal_id_encoded',
 'email_plus_homepage_encoded',
 'city_region_encoded',
 'op_area_encoded',
 'center_type_encoded',
 'category_encoded',
 'cuisine_encoded',
 'big_diff_encoded',
 'emailer_for_promotion_encoded',
 'homepage_featured_encoded',
 'city_code_encoded',
 'region_code_encoded']

In [338]:
categorical_columns = ['center_id_encoded',
                       'meal_id_encoded',
                       'email_plus_homepage_encoded',
                       'city_region_encoded',
                       'op_area_encoded',
                       'center_type_encoded',
                       'category_encoded',
                       'cuisine_encoded',
                       'big_diff_encoded'                       
                     ]

In [339]:
numerical_columns = [col for col in feature_name if col not in categorical_columns]

In [340]:
#numerical_columns.remove('checkout_price')
#numerical_columns.remove('base_price')

In [341]:
numerical_columns

['week',
 'checkout_price',
 'base_price',
 'price_diff',
 'price_diff_percent',
 'week_sin',
 'week_cos',
 'week_sin_1',
 'week_cos_1',
 'emailer_for_promotion_encoded',
 'homepage_featured_encoded',
 'city_code_encoded',
 'region_code_encoded']

* train set을  80 :20(train : valid) 으로 나눠 학습실행

(Feature Engineering) 'num_orders'의 값을 log transform

In [342]:
X = df_train[categorical_columns + numerical_columns]
#y = df_train['num_orders']
y = np.log1p(df_train['num_orders'])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                    test_size=0.02, 
                                                    shuffle=False)

* (parameter tuning) Grid search 를 통해 최적의 파라미터를 계산함(1차 feature eng. & parameter tuning에서 산출함)
* {'colsample_bytree': 0.4, 'min_child_samples': 5, 'num_leaves': 127'}

In [343]:
params = {'boosting_type' : 'gbdt',
          'objective': 'regression',
          'num_leaves':127,
          'learning_rate':0.01,
          'colsample_bytree': 0.4,
          'min_child_samples': 5,
          'n_estimators':30000,
          'max_depth':20,
          'metric':'rmse',
          }

### LightGBM Modeling

* Gradient Boosting Decision Tree
* Ensemble

참고 : https://lsjsj92.tistory.com/548


In [344]:
model = lgb.LGBMRegressor(**params)

In [345]:
model

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.4,
              importance_type='split', learning_rate=0.01, max_depth=20,
              metric='rmse', min_child_samples=5, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=30000, n_jobs=-1, num_leaves=127,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [346]:
params_fit = {'early_stopping_rounds':1000,
             'feature_name':numerical_columns+categorical_columns,
             'categorical_feature':categorical_columns,
             'eval_set':[(X_train,y_train), (X_valid, y_valid)]
             }

In [347]:
model.fit(X_train, y_train, **params_fit, verbose=500)

Training until validation scores don't improve for 1000 rounds.
[500]	training's rmse: 0.515178	valid_1's rmse: 0.52932
[1000]	training's rmse: 0.477929	valid_1's rmse: 0.496482
[1500]	training's rmse: 0.462406	valid_1's rmse: 0.485806
[2000]	training's rmse: 0.453441	valid_1's rmse: 0.479231
[2500]	training's rmse: 0.446808	valid_1's rmse: 0.47456
[3000]	training's rmse: 0.441005	valid_1's rmse: 0.471949
[3500]	training's rmse: 0.435867	valid_1's rmse: 0.469616
[4000]	training's rmse: 0.431734	valid_1's rmse: 0.468094
[4500]	training's rmse: 0.427848	valid_1's rmse: 0.466585
[5000]	training's rmse: 0.424254	valid_1's rmse: 0.465668
[5500]	training's rmse: 0.420992	valid_1's rmse: 0.464441
[6000]	training's rmse: 0.4179	valid_1's rmse: 0.463878
[6500]	training's rmse: 0.415036	valid_1's rmse: 0.463233
[7000]	training's rmse: 0.41227	valid_1's rmse: 0.462443
[7500]	training's rmse: 0.40958	valid_1's rmse: 0.461812
[8000]	training's rmse: 0.407024	valid_1's rmse: 0.461283
[8500]	training

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.4,
              importance_type='split', learning_rate=0.01, max_depth=20,
              metric='rmse', min_child_samples=5, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=30000, n_jobs=-1, num_leaves=127,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [350]:
X = df_test[categorical_columns + numerical_columns]

In [351]:
pred = model.predict(X)

In [352]:
pred = np.expm1(pred)

In [353]:
submission_df = df_test.copy()
submission_df['num_orders'] = pred
submission_df = submission_df[['id', 'num_orders']]
submission_df.to_csv('submission_simple_plus_cosin_sumcategory_pricediff.csv', index=False)

In [354]:
submission_df[submission_df['num_orders']<1]

,id,num_orders


## Result
* Baseline modeling : score = 60.6642916161, 1,344명중 521등, 상위 38% rank
* 1차 피쳐엔지니어링 & parameter tuning : score = 55.5030926505, 1,344명중 267등, 상위 19% rank
* 2차 피쳐엔지니어링 & parameter tuning : score = 52.6740812166, 1,344명중 112등, 상위 8% rank